# Sentiment Analysis using Text Mining

This Algorithm uses the reviews data of the clients and Analyses the review as positive or negative analysis

the data set for this Algorithm is given in the repository as "Amazon_Unlocked_Mobile.csv"

In [4]:
import pandas as pd
import numpy as np

df=pd.read_csv('Amazon_Unlocked_Mobile.csv')
df.shape
df=df.sample(frac=0.1,random_state=10)
print(df.shape)
df.head()

(41384, 6)


,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
394349,Sony XPERIA Z2 D6503 FACTORY UNLOCKED Internat...,NaN,244.95,5,Very good one! Better than Samsung S and iphon...,0.0
34377,Apple iPhone 5c 8GB (Pink) - Verizon Wireless,Apple,194.99,1,"The phone needed a SIM card, would have been n...",1.0
248521,Motorola Droid RAZR MAXX XT912 M Verizon Smart...,Motorola,174.99,5,I was 3 months away from my upgrade and my Str...,3.0
167661,CNPGD [U.S. Office Extended Warranty] Smartwat...,CNPGD,49.99,1,an experience i want to forget,0.0
73287,Apple iPhone 7 Unlocked Phone 256 GB - US Vers...,Apple,922.00,5,GREAT PHONE WORK ACCORDING MY EXPECTATIONS.,1.0


In [5]:
df.dropna(inplace=True)
df=df[df['Rating']!=3]

In [6]:
df['Positively Rated']=np.where(df['Rating']>3,1,0)
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated
34377,Apple iPhone 5c 8GB (Pink) - Verizon Wireless,Apple,194.99,1,"The phone needed a SIM card, would have been n...",1.0,0
248521,Motorola Droid RAZR MAXX XT912 M Verizon Smart...,Motorola,174.99,5,I was 3 months away from my upgrade and my Str...,3.0,1
167661,CNPGD [U.S. Office Extended Warranty] Smartwat...,CNPGD,49.99,1,an experience i want to forget,0.0,0
73287,Apple iPhone 7 Unlocked Phone 256 GB - US Vers...,Apple,922.00,5,GREAT PHONE WORK ACCORDING MY EXPECTATIONS.,1.0,1
277158,Nokia N8 Unlocked GSM Touch Screen Phone Featu...,Nokia,95.00,5,I fell in love with this phone because it did ...,0.0,1


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,t_test=train_test_split(df['Reviews'],df['Positively Rated'],random_state=0)

In [8]:
print('X_train first entry:\n\n', X_train.iloc[0])
print('\n\nX_train shape: ', X_train.shape)

X_train first entry:

 Everything about it is awesome!


X_train shape:  (23052,)


# Count Vectorizer/n-grams

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)

In [10]:
vect.get_feature_names()[::2000]

['00',
 'and verizon',
 'bought new',
 'couldnt',
 'fails',
 'great value',
 'is must',
 'love nokia',
 'not been',
 'performance for',
 'ref',
 'some areas',
 'the pin',
 'understood',
 'will replace']

In [11]:
len(vect.get_feature_names())

29072

In [12]:
x_train_vectorized=vect.transform(X_train)
x_train_vectorized

<23052x29072 sparse matrix of type '<class 'numpy.int64'>'
	with 1114538 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.linear_model import LogisticRegression

In [14]:
model=LogisticRegression()
model.fit(x_train_vectorized,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
pred=model.predict(vect.transform(X_test))

from sklearn.metrics import roc_auc_score
print('AUC: ', roc_auc_score(t_test, pred))

AUC:  0.91106617946


In [16]:
feature_names = np.array(vect.get_feature_names())

In [17]:
sorted_coef_index = model.coef_[0].argsort()

In [18]:
print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['no good' 'junk' 'poor' 'slow' 'worst' 'broken' 'not good' 'terrible'
 'defective' 'horrible']

Largest Coefs: 
['excellent' 'excelente' 'excelent' 'perfect' 'great' 'love' 'awesome'
 'no problems' 'good' 'best']


In [19]:
print(model.predict(vect.transform(['not bad phone is working',
                                    'an issue, phone is not in working'])))

[1 0]


Here,
1 is for the Positive review
2 is for the Negative review
